In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [3]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 74.9MB/s]


In [4]:
#  SmartCity Traffic Analyzer Web App with YOLOv8 + Route Optimization

import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np
import networkx as nx
from PIL import Image

# Load YOLOv8 model
model = YOLO("yolov8n.pt")

def analyze_traffic(image):
    # Save uploaded image
    image_path = "input.jpg"
    image.save(image_path)

    # Run YOLOv8 prediction
    results = model.predict(source=image_path, conf=0.4, save=False)
    boxes = results[0].boxes
    names = results[0].names
    vehicle_classes = ['car', 'truck', 'bus', 'motorbike']

    # Count vehicles
    vehicle_count = 0
    for i in range(len(boxes.cls)):
        class_name = names[int(boxes.cls[i])]
        if class_name in vehicle_classes:
            vehicle_count += 1

    # Traffic level logic
    if vehicle_count > 15:
        level = "High"
        color = (255, 0, 0)  # Red
        color_name = "Red"
        peak = "more than one hour"
        weight = 15
    elif 6 <= vehicle_count <= 15:
        level = "Medium"
        color = (255, 255, 0)  # Yellow
        color_name = "Yellow"
        peak = "between half an hour to one hour"
        weight = 10
    else:
        level = "Low"
        color = (0, 255, 0)  # Green
        color_name = "Green"
        peak = "less than half an hour"
        weight = 5

    # Heatmap overlay
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    overlay = img.copy()
    cv2.rectangle(overlay, (0, 0), (img.shape[1], img.shape[0]), color, -1)
    heatmap = cv2.addWeighted(overlay, 0.2, img, 0.8, 0)
    out_img = Image.fromarray(heatmap)

    # Add color display using emoji
    color_display = {
        "Red": "🟥",
        "Yellow": "🟨",
        "Green": "🟩"
    }

    result_text = (
        f"{color_display[color_name]} Traffic Level: {level}\n"
        f" Detected Vehicles: {vehicle_count}\n"
        f" Peak Hours: {peak}"
    )

    return out_img, result_text

def optimize_route(start, end):
    G = nx.Graph()
    # Simulated road network
    G.add_edge("A", "B", weight=5)
    G.add_edge("B", "C", weight=15)
    G.add_edge("A", "C", weight=10)
    G.add_edge("C", "D", weight=3)
    G.add_edge("B", "D", weight=12)

    try:
        path = nx.shortest_path(G, source=start, target=end, weight='weight')
        cost = nx.shortest_path_length(G, source=start, target=end, weight='weight')
        return f" Shortest path: {' → '.join(path)}\n💡 Total Congestion Cost: {cost}"
    except nx.NetworkXNoPath:
        return " No route available between selected points."

# Gradio interface
image_input = gr.Image(type="pil", label="Upload Traffic Image")
city_nodes = ["A", "B", "C", "D"]

iface = gr.Interface(
    fn=analyze_traffic,
    inputs=image_input,
    outputs=[gr.Image(label="Heatmap Result"), gr.Text(label="Analysis Result")],
    title="🚦 SmartCity Traffic Analyzer"
)

route_iface = gr.Interface(
    fn=optimize_route,
    inputs=[gr.Dropdown(city_nodes, label="From"), gr.Dropdown(city_nodes, label="To")],
    outputs=gr.Text(label="Optimized Route Result"),
    title="🛣️ Smart Route Optimizer"
)

app = gr.TabbedInterface([iface, route_iface], ["Traffic Detection", "Route Optimization"])

if __name__ == '__main__':
    app.launch(debug=False)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1ed6eb5102635b47d1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
